In [1]:
import os
import pandas as pd
import numpy as np
import ipywidgets as widgets

import bqplot.pyplot as plt
import bqplot as bq

## Image

In [2]:
image_path = os.path.abspath('image_nagra_nab14-17.png')

with open(image_path, 'rb') as f:
    raw_image = f.read()
ipyimage = widgets.Image(value=raw_image, format='png')
ipyimage

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x06\x86\x00\x00\x03\x1b\x08\x06\x00\x00\x00`\xdc\'\x…

## With pyplot-API of bqplot

In [3]:
plt.figure(padding_y=0)
axes_options = {'x': {'visible': False}, 'y': {'visible': False}}
plt.imshow(image_path, 'filename')
plt.show()

### Display inside a bqplot Figure

In [4]:
fig = plt.figure(title='Cross-Section', padding_x=0, padding_y=0)
image = plt.imshow(ipyimage, 'widget')
fig

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top':…

# Scatter Chart - Pyplot API

In [5]:
# data generation
price_data = pd.DataFrame(np.cumsum(np.random.randn(150, 2).dot([[1.0, -0.8], [-0.8, 1.0]]), axis=0) + 100,
                          columns=['Security 1', 'Security 2'], index=pd.date_range(start='01-01-2007', periods=150))
size = 100
np.random.seed(0)
x_data = range(size)
y_data = np.cumsum(np.random.randn(size) * 100.0)
ord_keys = np.array(['A', 'B', 'C', 'D', 'E', 'F'])
ordinal_data = np.random.randint(5, size=size)

In [6]:
# Scatter plot where you can move points and have an updated mean
fig2 = plt.figure()
scat = plt.scatter(x_data[:10], y_data[:10], colors=['SeaGreen'], 
                   enable_move=True, restrict_y=True)
lin = plt.plot([], [], line_style='dotted', colors=['DodgerBlue'])

def update_mean(change=10):
    with lin.hold_sync():
        lin.x = [np.min(scat.x), np.max(scat.x)]
        lin.y = [np.mean(scat.y), np.mean(scat.y)]
        
update_mean()

# update line on change of x or y of scatter
scat.observe(update_mean, names=['x'])
scat.observe(update_mean, names=['y'])

fig2

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top':…

In [7]:
# Scatter plot where you can add and move points 
fig2 = plt.figure()
scat = plt.scatter(x_data[:10], y_data[:10], colors=['SeaGreen'], 
                   enable_move=True, restrict_y=True)
lin = plt.plot([], [], line_style='dotted', colors=['DodgerBlue'])

def update_mean(change=None):
    with lin.hold_sync():
        lin.x = [np.min(scat.x), np.max(scat.x)]
        lin.y = [np.mean(scat.y), np.mean(scat.y)]
    with scat.hold_sync():
        scat.interactions = {'click': 'add'}
update_mean()

# update line on change of x or y of scatter
scat.observe(update_mean, names=['x'])
scat.observe(update_mean, names=['y'])

fig2

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top':…

In [8]:
# Image and scatter in one
fig3 = plt.figure(padding_x=0, padding_y=0)

# xz dimensions in km
xdim = np.array([0, 30.])
ydim = np.array([-6.5, 1.])

plt.scales(scales = {'x': bq.LinearScale(min=np.min(xdim), max=np.max(xdim)), 
          'y': bq.LinearScale(min=np.min(ydim), max=np.max(ydim))})

image = plt.imshow(x=xdim, y=ydim, image=ipyimage, format='widget')


scat = plt.scatter(x=[], y=[], colors=['SeaGreen'], 
                   enable_move=True, restrict_y=False)

def add_points(change=None):
    with scat.hold_sync():
        scat.interactions = {'click': 'add'}
add_points()
fig3

Figure(axes=[Axis(scale=LinearScale(max=30.0, min=0.0)), Axis(orientation='vertical', scale=LinearScale(max=1.…

In [10]:
# Image and scatter in one with buttons
fig4 = plt.figure(padding_x=0, padding_y=0)

# dict for layers and colors - replace with architecture in gempy
cord=['DodgerBlue', 'DarkSlateGray', 'Yellow', 'HotPink', 'OrangeRed']
layers = ['GrabenFill', 'PostGraben1', 'Postgraben2', 'Jura', 'MainFault']
laycol = dict(zip(layers, cord))

# buttons
add_btn = widgets.Button(description='add', button_style='success')
delete_btn = widgets.Button(description='delete', button_style='danger')
series_btn = widgets.Button(description='save to file', button_style='warning')
layers_btn = widgets.Dropdown(description='unit', options=layers)

# xz dimensions in km
xdim = np.array([0, 30.])
ydim = np.array([-6.5, 1.])

plt.scales(scales = {'x': bq.LinearScale(min=np.min(xdim), max=np.max(xdim)), 
          'y': bq.LinearScale(min=np.min(ydim), max=np.max(ydim))})

image = plt.imshow(x=xdim, y=ydim, image=ipyimage, format='widget')


scat = plt.scatter(x=[], y=[], colors=['SeaGreen'], 
                   enable_move=True, restrict_y=False)

plt.xlabel('x [km]')
plt.ylabel('z [km]')

def add_points():
    with image.hold_sync():
        scat.interactions = {'click': 'add'}

def delete_points():
    with scat.hold_sync():
        scat.interactions = {'click': 'delete'}
        
def save_fid():
    # update the y attribute of line mark
    xy = np.stack([scat.x*1000,scat.y*1000],axis=1)
    
    np.savetxt(layers_btn.value+'.csv', xy, fmt='%.2f', header='x[m],z[m]', delimiter=',')
# create a callback which updates the plot when dropdown item is selected, caution, deletes all points
# new scat instance has to be called each time this function is called
def change_interface(*args):
    selected_ticker = layers_btn.value
    scat.x = []
    scat.x = []
    # update the y attribute of the mark by selecting 
    # the column from the price data frame
    scat.colors = [laycol[selected_ticker]]
    

# register the callback by using the 'observe' method
layers_btn.observe(change_interface, 'value')
# register the on_click function
add_btn.on_click(lambda btn: add_points())
delete_btn.on_click(lambda btn: delete_points())
series_btn.on_click(lambda btn: save_fid())

# stack button and figure using VBox
widgets.VBox([fig4, widgets.HBox([add_btn, delete_btn, series_btn, layers_btn])])



Problem is, this does not work well. New points cannot be added on top of the image.  
One has to click on the side of the plot (next to the image) and drag the point to its destination.  
Needs to be fixed...maybe by loading the image differently?